In [ ]:
import sys
sys.path.append("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm")

from biollm.utils.utils import load_config

import scanpy as sc
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [2]:
import os
os.chdir("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/tutorials/zero-shot")

#### Get Gene Expression Embedding

##### scFoundation

In [1]:
from biollm.base.load_scfoundation import LoadScfoundation

config_file = './configs/scfoundation_gene-expression_emb.toml'
configs = load_config(config_file)

obj = LoadScfoundation(configs)
print(obj.args)

adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["gene_symbols"].tolist()

obj.model = obj.model.to(configs.device)
emb = obj.get_embedding(configs.emb_type, gene_ids=None, adata=adata)
print('embedding shape:', emb.shape)

vocab = {v :k for k, v in obj.get_gene2idx().items()}
data = pd.DataFrame(emb)
data.index = [vocab[idx] for idx in data.index.tolist()]
data = data.loc[data.index.isin(adata.var_names), :]
scf_gene_emb = data

ModuleNotFoundError: No module named 'biollm'

In [ ]:
import os
import pickle

gene_name_file = os.path.join(configs.output_dir, "scf_gene_names.pkl")
with open(gene_name_file, 'wb') as file:
    pickle.dump(scf_gene_emb.index.tolist(), file)
    
gene_emb_file = os.path.join(configs.output_dir, "scf_gene_emb.pkl")
with open(gene_emb_file, 'wb') as file:
    pickle.dump(np.array(scf_gene_emb), file)

##### scMamba

In [3]:
from biollm.base.load_mamba import LoadScmamba

config_file = './configs/scmamba_gene-expression_emb.toml'
configs = load_config(config_file)

obj = LoadScmamba(configs)
print(obj.args)

adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["gene_symbols"].tolist()
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
adata = obj.preprocess_adata(adata)
obj.args['max_seq_len'] = adata.shape[1]

file_name = os.path.basename(configs.input_file)

if 'gene_name' not in adata.var:
    adata.var['gene_name'] = adata.var.index.values
    
gene_ids = [obj.get_gene2idx().get(key) for key in adata.var_names]
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)

obj.model = obj.model.to(configs.device)
emb, gene_names = obj.get_embedding(obj.args.emb_type, adata, gene_ids)
emb = pd.DataFrame(emb, index = gene_names)

mamba_gene_emb = emb

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/lightning/fabric/plugins/environments/xla.py:18: DeprecationWarning: `ModuleAvailableCache` is a special case of `RequirementCache`. Please use `RequirementCache(module=...)` instead.
  from lightning.fabric.accelerators.tpu import _XLA_AVAILABLE, TPUAccelerator
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pk

Using simple batchnorm instead of domain specific batchnorm


2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params encoder.embedding.weight with shape torch.Size([60697, 512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params encoder.enc_norm.weight with shape torch.Size([512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params encoder.enc_norm.bias with shape torch.Size([512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder.linear1.weight with shape torch.Size([512, 1])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder.linear1.bias with shape torch.Size([512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder.linear2.weight with shape torch.Size([512, 512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder.linear2.bias with shape torch.Size([512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder.norm.weight with shape torch.Size([512])
2024-06-19 10:22:15-load_llm[line-40]-INFO: Loading params value_encoder

Munch({'model_used': 'scmamba', 'emb_type': 'gene-expression', 'model_file': '../../case/models/scmamba/best_model.pt', 'model_param_file': '../../case/models/scmamba/args.json', 'vocab_file': '../../case/models/scmamba/vocab.json', 'input_file': '/home/share/huadjyin/home/s_huluni/cellxgene/h5ads/d7d7e89c-c93a-422d-8958-9b4a90b69558.h5ad', 'distributed': False, 'do_pretrain': False, 'output_dir': '../../case/result/zero-shot/', 'device': 'cuda:3', 'include_zero_gene': True, 'hvg_number': 0, 'data_is_raw': False, 'input_style': 'binned', 'task': 'Embedding', 'pad_token': '<pad>', 'pad_value': -2, 'batch_size': 32, 'mask_ratio': 0, 'CLS': False, 'ADV': False, 'CCE': False, 'MVC': False, 'ECS': False, 'data_name': 'cellxgene', 'model_name': 'mamba', 'run_name': 'gst_ori_initemb', 'single_gpu': False, 'do_train': True, 'GEPC': False, 'epochs': 5, 'seed': 1927, 'dropout': 0.2, 'lr': 1e-06, 'num_workers': 0, 'log_interval': 1000, 'save_eval_interval': 5, 'schedule_ratio': 0.9, 'amp': True, 

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type

In [ ]:
import os
import pickle

gene_name_file = os.path.join(configs.output_dir, "mamba_gene_names.pkl")
with open(gene_name_file, 'wb') as file:
    pickle.dump(mamba_gene_emb.index.tolist(), file)
    
gene_emb_file = os.path.join(configs.output_dir, "mamba_gene_emb.pkl")
with open(gene_emb_file, 'wb') as file:
    pickle.dump(np.array(mamba_gene_emb), file)

##### scBERT

In [ ]:
from biollm.base.load_scbert import LoadScbert

config_file = './configs/scbert_gene-expression_emb.toml'
configs = load_config(config_file)
obj = LoadScbert(configs)
print(obj.args)

gene_ids = list(obj.get_gene2idx().values())
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)
obj.model = obj.model.to(configs.device)
adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["gene_symbols"].tolist()
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
emb = obj.get_embedding(configs.emb_type, adata=adata)
print('embedding shape:', emb.shape)

vocab = {v :k for k, v in obj.get_gene2idx().items()}
data = pd.DataFrame(emb)
data.index = [vocab[idx] for idx in data.index.tolist()]
data = data.loc[data.index.isin(adata.var_names), :]
scb_gene_emb = data

In [ ]:
import os
import pickle

gene_name_file = os.path.join(configs.output_dir, "scb_gene_names.pkl")
with open(gene_name_file, 'wb') as file:
    pickle.dump(scb_gene_emb.index.tolist(), file)
    
gene_emb_file = os.path.join(configs.output_dir, "scb_gene_emb.pkl")
with open(gene_emb_file, 'wb') as file:
    pickle.dump(np.array(scb_gene_emb), file)

##### scGPT

In [ ]:
from biollm.base.load_scgpt import LoadScgpt

config_file = './configs/scgpt_gene-expression_emb.toml'
configs = load_config(config_file)

adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["gene_symbols"].tolist()
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
adata.var["gene_name"] = adata.var["feature_name"]
adata.obs["celltype_id"] = adata.obs["cell_type"].cat.codes
adata.obs["batch_id"] = 0

obj = LoadScgpt(configs)
adata = adata[:, adata.var_names.isin(obj.get_gene2idx().keys())].copy()
configs.max_seq_len = adata.var.shape[0] + 1
obj = LoadScgpt(configs)
print(obj.args)

obj.model = obj.model.to(configs.device)

emb = obj.get_embedding(configs.emb_type, adata=adata)
print('embedding shape:', emb.shape)

data = pd.DataFrame(emb)
data.index = adata.var_names
scg_gene_emb = data

In [ ]:
import os
import pickle

gene_name_file = os.path.join(configs.output_dir, "scg_gene_names.pkl")
with open(gene_name_file, 'wb') as file:
    pickle.dump(scg_gene_emb.index.tolist(), file)
    
gene_emb_file = os.path.join(configs.output_dir, "scg_gene_emb.pkl")
with open(gene_emb_file, 'wb') as file:
    pickle.dump(np.array(scg_gene_emb), file)

##### GeneFormer


In [4]:
from biollm.base.load_geneformer import LoadGeneformer

config_file = './configs/geneformer_gene-expression_emb.toml'
configs = load_config(config_file)

obj = LoadGeneformer(configs)
print(obj.args)

adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["feature_name"]
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
file_name = os.path.basename(configs.input_file)
obj.model = obj.model.to(configs.device)
emb, gene_ids = obj.get_embedding(obj.args.emb_type, adata=adata)
print('embedding shape:', emb.shape)

vocab = {v:k for k,v in  obj.get_gene2idx().items()}
gene_names = [vocab[id] for id in gene_ids]

gf_gene_emb = pd.DataFrame(emb, index=gene_names)

Munch({'model_used': 'geneformer', 'model_type': 'Pretrained', 'model_file': '/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/models/geneformer', 'vocab_file': '/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/models/geneformer/gene_vocab.json', 'n_bins': 100, 'input_file': '/home/share/huadjyin/home/s_huluni/cellxgene/h5ads/d7d7e89c-c93a-422d-8958-9b4a90b69558.h5ad', 'output_dir': '../../case/result/zero-shot/', 'device': 'cuda:0', 'emb_type': 'gene-expression', 'encoder_dims': 768, 'encoder_heads': 12, 'encoder_depth': 12})


/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/anndata/_core/anndata.py:956: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['RP11-34P13.7', 'DDX11L17', 'WASH9P', 'AP006222.1', 'RP11-206L10.17']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/rel

AnnData has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map (num_proc=16):   0%|          | 0/16901 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/16901 [00:00<?, ? examples/s]

  0%|          | 0/529 [00:00<?, ?it/s]

embedding shape: (1363, 256)


In [5]:
import os
import pickle

gene_name_file = os.path.join(configs.output_dir, "gf_gene_names.pkl")
with open(gene_name_file, 'wb') as file:
    pickle.dump(gf_gene_emb.index.tolist(), file)
    
gene_emb_file = os.path.join(configs.output_dir, "gf_gene_emb.pkl")
with open(gene_emb_file, 'wb') as file:
    pickle.dump(np.array(gf_gene_emb), file)

##### PCA

In [10]:
adata = sc.read_h5ad(configs.input_file)
adata.var_names = adata.var["gene_symbols"].tolist()
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
sc.pp.scale(adata)
sc.pp.pca(adata)
emb = adata.varm["PCs"]
pca_gene_emb = pd.DataFrame(emb, index = adata.var_names)

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type

#### Get GRN Graph

In [4]:
import hnswlib

def construct_grn_graph(data):
    
    gene_dict = dict(zip(range(len(data)), data.index))
    data = np.array(data)
    
    num_elements = data.shape[0]
    dim = data.shape[1]
    ids = np.arange(num_elements)
    
    # Declaring index
    p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip
    
    # Initializing index - the maximum number of elements should be known beforehand
    p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
    
    # Element insertion (can be called several times):
    p.add_items(data, ids)
    
    # Controlling the recall by setting ef:
    p.set_ef(100) # ef should always be > k
    
    # Query dataset, k - number of the closest elements (returns 2 numpy arrays)
    k_neighbors = 100
    labels, _ = p.knn_query(data, k = k_neighbors + 1)
    
    graph = labels[:, 1:]
    edgeDF = np.stack([np.repeat(np.arange(graph.shape[0]), graph.shape[1]),
                      graph.ravel()], axis=1, dtype=int)
    
    
    edgeDF = pd.DataFrame(edgeDF)
    edgeDF.columns = ["Gene1", "Gene2"]
    
    edgeDF["Gene1"] = [gene_dict[key] for key in edgeDF["Gene1"]]
    edgeDF["Gene2"] = [gene_dict[key] for key in edgeDF["Gene2"]]
    
    return edgeDF

#### GRN evaluation

In [26]:
import pyarrow.feather as feather
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import pandas as pd

def process_edges(df):
    
    pos_edges = []
    neg_edges = []

    for i, row in tqdm(df.iterrows(), total=len(df) , desc="Processing TFs"):
        pos_edges.append(row.sort_values(ascending=True)[:50].index.tolist())
        neg_edges.append(row.sort_values(ascending=False)[:50].index.tolist())

    pos_edges = pd.DataFrame(pos_edges, index=df.index)
    pos_edges = pos_edges.melt(ignore_index=False)
    pos_edges["Gene1"] = pos_edges.index.tolist()
    pos_edges["Gene2"] = pos_edges["value"].tolist()
    pos_edges = pos_edges.loc[:, ["Gene1", "Gene2"]]
    pos_edges["class"] = 1

    neg_edges = pd.DataFrame(neg_edges, index=df.index)
    neg_edges = neg_edges.melt(ignore_index=False)
    neg_edges["Gene1"] = neg_edges.index.tolist()
    neg_edges["Gene2"] = neg_edges["value"].tolist()
    neg_edges = neg_edges.loc[:, ["Gene1", "Gene2"]]
    neg_edges["class"] = 0

    return pd.concat([pos_edges, neg_edges], axis=0)

def evaluate_grn(edgeDF, df, motif_df):
    df.index = df["motifs"].tolist()
    df = df.drop("motifs", axis=1)

    motif_dict = dict(zip(motif_df.index, motif_df["gene_name"]))
    df = df.loc[df.index.isin(motif_dict.keys()), :]
    df.index = [motif_dict[key] for key in df.index]
    
    gene_list = list(set(edgeDF["Gene1"]).union(edgeDF["Gene2"]))
    df = df.loc[df.index.isin(gene_list), df.columns.isin(gene_list)]

    true_edge_df = process_edges(df)
    true_edge = true_edge_df.set_index(["Gene1", "Gene2"]).index
    pred_edge = edgeDF.set_index(["Gene1", "Gene2"]).index

    out_df = pd.DataFrame(index=true_edge)
    out_df["TrueEdges"] = true_edge_df["class"].tolist()
    out_df["PredEdges"] = 0
    out_df["PredEdges"][out_df.index.isin(pred_edge)] = 1
    print((out_df.shape, sum(out_df["PredEdges"])))
    
    fpr, tpr, thresholds = roc_curve(y_true=out_df['TrueEdges'].tolist(), y_score=out_df['PredEdges'].tolist(), pos_label=1)
    prec, recall, thresholds = precision_recall_curve(y_true=out_df['TrueEdges'].tolist(), probas_pred=out_df['PredEdges'].tolist(), pos_label=1)

    return prec, recall, fpr, tpr, auc(recall, prec), auc(fpr, tpr)

def grn_eval(edgeDF):
    df = feather.read_feather('/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings.feather')
    motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
    return evaluate_grn(edgeDF, df, motif_df)

#### runs

In [8]:
import pickle

with open("../../case/result/zero-shot/"+"gf_gene_emb.pkl", 'rb') as f:
    gf_gene_emb = pickle.load(f)
    
with open("../../case/result/zero-shot/"+"gf_gene_names.pkl", 'rb') as f:
    gene_names = pickle.load(f)

gf_gene_emb = pd.DataFrame(gf_gene_emb, index=gene_names)

with open("../../case/result/zero-shot/"+"scb_gene_emb.pkl", 'rb') as f:
    scb_gene_emb = pickle.load(f)
    
with open("../../case/result/zero-shot/"+"scb_gene_names.pkl", 'rb') as f:
    gene_names = pickle.load(f)

scb_gene_emb = pd.DataFrame(scb_gene_emb, index=gene_names)

with open("../../case/result/zero-shot/"+"mamba_gene_emb.pkl", 'rb') as f:
    mamba_gene_emb = pickle.load(f)
    
with open("../../case/result/zero-shot/"+"mamba_gene_names.pkl", 'rb') as f:
    gene_names = pickle.load(f)

mamba_gene_emb = pd.DataFrame(mamba_gene_emb, index=gene_names)

with open("../../case/result/zero-shot/"+"scg_gene_emb.pkl", 'rb') as f:
    scg_gene_emb = pickle.load(f)
    
with open("../../case/result/zero-shot/"+"scg_gene_names.pkl", 'rb') as f:
    gene_names = pickle.load(f)

scg_gene_emb = pd.DataFrame(scg_gene_emb, index=gene_names)

with open("../../case/result/zero-shot/"+"scf_gene_emb.pkl", 'rb') as f:
    scf_gene_emb = pickle.load(f)
    
with open("../../case/result/zero-shot/"+"scf_gene_names.pkl", 'rb') as f:
    gene_names = pickle.load(f)

scf_gene_emb = pd.DataFrame(scf_gene_emb, index=gene_names)

In [21]:
import numpy as np
import pandas as pd
import json

emb = np.load("/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/bio_model/biollm/case/models/scbert/gene2vec_16906.npy")

with open('/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/bio_model/biollm/case/models/scbert/gene_vocab.json', 'r') as f:
    vocab = json.load(f)

gene2vec_emb = pd.DataFrame(emb, index = list(vocab.keys()))
gene2vec_emb = gene2vec_emb.loc[gene2vec_emb.index.isin(adata.var_names), :]

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [27]:
emb_dict = {"scFoundation":scf_gene_emb,
            "scBERT":scb_gene_emb,
            "scGPT":scg_gene_emb,
            "scMamba":mamba_gene_emb,
            "GeneFormer":gf_gene_emb,
            "PCA":pca_gene_emb,
            'gene2vec': gene2vec_emb
            }

In [28]:
eval_model = {}

for k, v in emb_dict.items():
    edge_df = construct_grn_graph(v)
    eval_res = grn_eval(edge_df)
    eval_res = {"AUPRC":eval_res[4], "AUROC":eval_res[5]}
    eval_model.update({k:eval_res})

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/tmp/ipykernel_408806/3569791768.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more informati

((144700, 2), 11344)


/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/tmp/ipykernel_408806/3569791768.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more informati

((144700, 2), 10777)


/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/tmp/ipykernel_408806/3569791768.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more informati

...

/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/tmp/ipykernel_408806/3569791768.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more informati

((145000, 2), 7855)


/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/tmp/ipykernel_408806/3569791768.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motif_df = pd.read_csv("/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/case/ref/motifs-v9-nr.hgnc-m0.001-o0.0.tbl", sep='\t', index_col=0)
/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more informati

((144700, 2), 11351)


/home/share/huadjyin/home/s_huluni/anaconda3/envs/py39_torch/lib/python3.9/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [29]:
eval_model

{'scFoundation': {'AUPRC': 0.5261585299286594, 'AUROC': 0.501907394609537},
 'scBERT': {'AUPRC': 0.6585250580676206, 'AUROC': 0.5387906012439531},
 'scGPT': {'AUPRC': 0.5858104346880412, 'AUROC': 0.5161161022805805},
 'scMamba': {'AUPRC': 0.560668166463734, 'AUROC': 0.510463026952315},
 'GeneFormer': {'AUPRC': 0.5631247011817987, 'AUROC': 0.511969592259848},
 'PCA': {'AUPRC': 0.564037358150969, 'AUROC': 0.5103793103448275},
 'gene2vec': {'AUPRC': 0.5967821746462093, 'AUROC': 0.5224533517622668}}

In [30]:
import pickle

pkl_file = "/home/share/huadjyin/home/s_huluni/project/bio_model/biollm/biollm/evaluate/gene_setinfo.pkl"
# Load the pickle file
with open(pkl_file, "rb") as file:
    gene_pw_dict = pickle.load(file)

In [31]:
from biollm.evaluate.bm_metrices_grn import evaluate
import networkx as nx
eval_res = {}
for k, v in emb_dict.items():
    edge_df = construct_grn_graph(v)
    g = nx.from_pandas_edgelist(edge_df, source="Gene1", target="Gene2")
    res = evaluate(g, gene_pw_dict)
    eval_res.update({k:res})

In [32]:
eval_res

{'scFoundation': {'Modularity': 0.367546966266302,
  'Biological Index': 0.02107849522249016},
 'scBERT': {'Modularity': 0.3777987695897467,
  'Biological Index': 0.02106492046029973},
 'scGPT': {'Modularity': 0.5214893246362132,
  'Biological Index': 0.07417045168765554},
 'scMamba': {'Modularity': 0.6419021133541871,
  'Biological Index': 0.08803182961457973},
 'GeneFormer': {'Modularity': 0.19107282576386592,
  'Biological Index': 0.019754984032642676},
 'PCA': {'Modularity': 0.322332363133518,
  'Biological Index': 0.04959567069421787},
 'gene2vec': {'Modularity': 0.3825188901280674,
  'Biological Index': 0.023186796660255565}}